In [1]:
import networkx as nx
import numpy as np
import pandas as pd

import collections
import heapq
import math
import pickle
import random
from tqdm.notebook import tqdm
import time
import os

# 1 Реализация `NAMOA` и `BOA`
## 1.1 Общая часть

In [2]:
def read_edgelists(path_distance_data, path_time_data):
    graph = nx.Graph()
    V = None
    E = None
    with open(path_distance_data, 'r') as file_distance_data, open(path_time_data, 'r') as file_time_data:
        for line_distance, line_time in zip(file_distance_data, file_time_data):
            tokens_distance = line_distance.strip().split()
            tokens_time = line_time.strip().split()
            assert tokens_distance[0] == tokens_time[0]
            
            if tokens_distance[0] == 'p':
                V = int(tokens_distance[2])
                assert V == int(tokens_time[2])
                
                E = int(tokens_distance[3])
                assert E == int(tokens_time[3])
            elif tokens_distance[0] == 'a':
                E -= 1
                node1 = int(tokens_distance[1])
                assert node1 == int(tokens_time[1])
                
                node2 = int(tokens_distance[2])
                assert node2 == int(tokens_time[2])
                
                distance = int(tokens_distance[3])
                time = int(tokens_time[3])
                graph.add_edge(node1, node2, distance=distance, time=time)
        
        assert len(graph.nodes()) == V, f'Expected nodes:{V}, actual:{len(graph.nodes())}.'
        assert E == 0, f'Expected: all edges are read, actual:{E} edges aren\'t read.'
    
    return graph


class Node:
    def __init__(self, vertex, g, f, parent):
        super().__init__()
        self.vertex = vertex
        self.g = g
        self.f = f
        self.parent = parent
        self.tuple = (self.vertex, self.g, self.f, self.parent)
    
    def __eq__(self, other):
        return self.tuple == other.tuple
    
    def __hash__(self):
        return hash(self.tuple)
    
    def __lt__(self, other):
        return self.f < other.f
    
    def __repr__(self):
        return f'vertex={self.vertex} g={self.g} f={self.f} parent={self.parent}'


def dominates(p, q):
    return (p[0] < q[0] and p[1] <= q[1]) or (p[0] == q[0] and p[1] < q[1])

## 1.2 Реализация `NAMOA`

In [3]:
class OpenNAMOA:
    def __init__(self):
        super().__init__()
        self.queue = []
        self.valid_nodes = collections.defaultdict(set)
        self.n_invalid = 0
        self.max_size = 0
        self.max_size_invalid = 0
    
    def is_empty(self):
        return len(self.queue) == self.n_invalid
    
    def _get_valid_nodes(self, node):
        return self.valid_nodes[(node.vertex, node.g)]
    
    def push(self, node):
        self._get_valid_nodes(node).add(node)
        heapq.heappush(self.queue, [node.f, node])
        if len(self.queue) > self.max_size_invalid:
            self.max_size_invalid = len(self.queue)
        
        if len(self.queue) - self.n_invalid > self.max_size:
            self.max_size = len(self.queue) - self.n_invalid
    
    def remove_dominated(self, f):
        for _, node in self.queue:
            valid_nodes = self._get_valid_nodes(node)
            if node not in valid_nodes:
                continue
            
            if dominates(f, node.f):
                valid_nodes.remove(node)
                self.n_invalid += 1
    
    def pop(self):
        _, node = heapq.heappop(self.queue)
        valid_nodes = self._get_valid_nodes(node)
        while node not in valid_nodes:
            self.n_invalid -= 1
            _, node = heapq.heappop(self.queue)
            valid_nodes = self._get_valid_nodes(node)
        
        valid_nodes.remove(node)
        return node
    
    def invalidate(self, vertex, g_values):
        for g in g_values:
            valid_nodes = self.valid_nodes.pop((vertex, g), None)
            if valid_nodes is not None:
                self.n_invalid += len(valid_nodes)


def remove_dominated(g, g_set):
    to_remove = [g_dominated for g_dominated in g_set if dominates(g, g_dominated)]
    for g_dominated in to_remove:
        g_set.remove(g_dominated)
    
    return to_remove


def namoa(graph, start, goal):
    start_time = time.time()
    n_expansions = 0
    
    solutions = []
    h0 = nx.shortest_path_length(graph, source=goal, weight='distance')
    h1 = nx.shortest_path_length(graph, source=goal, weight='time')
    g_open = collections.defaultdict(set)
    g_close = collections.defaultdict(set)
    
    parent = {(0, 0): set()}
    g_open[start].add((0, 0))
    open_ = OpenNAMOA()
    open_.push(Node(vertex=start, g=(0, 0), f=(h0[start], h1[start]), parent=None))
    
    while not open_.is_empty():
        node = open_.pop()
        g_open[node.vertex].remove(node.g)
        g_close[node.vertex].add(node.g)
        if node.vertex == goal:
            solutions.append(node.g)
            open_.remove_dominated(node.f)
            continue
        
        n_expansions += 1
        for edge in graph.edges(node.vertex, data=True):
            next_vertex = edge[1]
            cost = (edge[2]['distance'], edge[2]['time'])
            next_g = (node.g[0] + cost[0], node.g[1] + cost[1])
            if next_g in g_open[next_vertex] or next_g in g_close[next_vertex]:
                parent[next_g].add(node.g)
                continue
            
            if any(dominates(g, next_g) for g in g_open[next_vertex]) \
                    or any(dominates(g, next_g) for g in g_close[next_vertex]):
                continue
            
            next_f = (next_g[0] + h0[next_vertex], next_g[1] + h1[next_vertex])
            if any(dominates(g, next_f) for g in solutions):
                continue
            
            to_remove = remove_dominated(next_g, g_open[next_vertex])
            open_.invalidate(next_vertex, to_remove)
            
            remove_dominated(next_g, g_close[next_vertex])
            parent[next_g] = set([node.g])
            g_open[next_vertex].add(next_g)
            open_.push(Node(next_vertex, next_g, next_f, parent=None))
    
    return {
        'solutions': solutions,
        'parent': parent,
        'n_expansions': n_expansions,
        'runtime': time.time() - start_time,
        'max_size': open_.max_size,
        'max_size_invalid': open_.max_size_invalid
    }

## 1.3 Реализация `BOA`

In [4]:
class OpenBOA:
    def __init__(self):
        super().__init__()
        self.queue = []
        self.max_size = 0
    
    def __len__(self):
        return len(self.queue)
    
    def __iter__(self):
        return iter(self.queue)
    
    def is_empty(self):
        return len(self.queue) == 0
    
    def push(self, node):
        heapq.heappush(self.queue, node)
        if len(self.queue) > self.max_size:
            self.max_size = len(self.queue)
    
    def pop(self):
        return heapq.heappop(self.queue)


def boa(graph, start, goal):
    start_time = time.time()
    n_expansions = 0
    solutions = []
    h0 = nx.shortest_path_length(graph, source=goal, weight='distance')
    h1 = nx.shortest_path_length(graph, source=goal, weight='time')
    g1_min = collections.defaultdict(lambda: math.inf)
    open_ = OpenBOA()
    open_.push(Node(vertex=start, g=(0, 0), f=(h0[start], h1[start]), parent=None))
    
    while not open_.is_empty():
        node = open_.pop()
        if node.g[1] >= g1_min[node.vertex] or node.f[1] >= g1_min[goal]:
            continue
        
        g1_min[node.vertex] = node.g[1]
        if node.vertex == goal:
            solutions.append(node)
            continue
        
        n_expansions += 1
        for edge in graph.edges(node.vertex, data=True):
            vertex = edge[1]
            cost = (edge[2]['distance'], edge[2]['time'])
            g = (node.g[0] + cost[0], node.g[1] + cost[1])
            f = (g[0] + h0[vertex], g[1] + h1[vertex])
            if g[1] >= g1_min[vertex] or f[1] >= g1_min[goal]:
                continue
            
            successor = Node(vertex, g, f, parent=node)
            open_.push(successor)
    
    return {
        'solutions': solutions,
        'n_expansions': n_expansions,
        'runtime': time.time() - start_time,
        'max_size': open_.max_size
    }

## 1.4 Наивный алгоритм

In [5]:
def naive_algorithm(graph, start):
    g_value = {vertex: set() for vertex in graph.nodes()}
    g_value[start].add((0, 0))
    open_ = collections.deque([start])
    
    while len(open_) > 0:
        vertex = open_.pop()
        for edge in graph.edges(vertex, data=True):
            next_vertex = edge[1]
            cost = (edge[2]['distance'], edge[2]['time'])
            update = False
            for g in g_value[vertex]:
                g_candidate = (g[0] + cost[0], g[1] + cost[1])
                if g_candidate in g_value[next_vertex] \
                        or any(dominates(next_g, g_candidate) for next_g in g_value[next_vertex]):
                    continue
                
                remove_dominated(g_candidate, g_value[next_vertex])
                g_value[next_vertex].add(g_candidate)
                update = True
            
            if update:
                open_.appendleft(next_vertex)
    
    return g_value

# 2 Проверка корректности реализаций `NAMOA` и `BOA`
##  2.1 Проверка на случайном графе из 1000 вершин

In [6]:
V = 1000
random_graph = nx.gnp_random_graph(V, 0.1)
for edge in random_graph.edges():
    random_graph[edge[0]][edge[1]]['distance'] = random.randint(1, 1000)
    random_graph[edge[0]][edge[1]]['time'] = random.randint(1, 1000)

print(f'E={len(random_graph.edges())}')

E=49779


In [7]:
%%time
source = 0
ground_truth = naive_algorithm(random_graph, source)
for goal in tqdm(random_graph.nodes()):
    if goal == source:
        continue
    
    boa_solutions = set([node.g for node in boa(random_graph, source, goal)['solutions']])
    namoa_solutions = set(namoa(random_graph, source, goal)['solutions'])
    if ground_truth[goal] != boa_solutions:
        raise AssertionError('BOA results are inconsistent with ground truth.')
    if ground_truth[goal] != namoa_solutions:
        raise AssertionError('NAMOA results are inconsistent with ground truth.')


CPU times: user 23min 57s, sys: 1.32 s, total: 23min 58s
Wall time: 23min 58s


##  2.2 Проверка на подграфе DIMACS из 10000 вершин

In [8]:
graph = read_edgelists('USA-road-d.NY.gr', 'USA-road-t.NY.gr')
print(f'E={len(graph.edges())}')
print(f'V={len(graph.nodes())}')
print(f'Mean degree:{np.mean([degree for node, degree in graph.degree()])}')

E=365050
V=264346
Mean degree:2.7619105263556096


In [9]:
%%time
source = 20000
V = 10000
nodes = [source] + [edge[1] for i, edge in enumerate(nx.bfs_edges(graph, source=source)) if i < V]
subgraph = graph.subgraph(nodes)
print(f'E={len(subgraph.edges())}')
print(f'V={len(subgraph.nodes())}')
print(f'Mean degree:{np.mean([degree for node, degree in subgraph.degree()])}')

ground_truth = naive_algorithm(subgraph, source)
for goal in tqdm(subgraph.nodes()):
    if goal == source:
        continue
    
    boa_solutions = set([node.g for node in boa(subgraph, source, goal)['solutions']])
    namoa_solutions = set(namoa(subgraph, source, goal)['solutions'])
    if ground_truth[goal] != boa_solutions:
        raise AssertionError('BOA results are inconsistent with ground truth.')
    if ground_truth[goal] != namoa_solutions:
        raise AssertionError('NAMOA results are inconsistent with ground truth.')

E=13582
V=10001
Mean degree:2.716128387161284



CPU times: user 1h 42min 59s, sys: 7.45 s, total: 1h 43min 7s
Wall time: 1h 43min 1s


# 3 Сравнение на `NAMOA` и `BOA` на данных DIMACS
- Работа алгоритмов сравнивалась на 100 случайно выбраных парах вершин `pairs.txt`
```python
import random
N_PAIRS = 100
graph = read_edgelists('USA-road-d.NY.gr', 'USA-road-t.NY.gr')
nodes = random.sample(list(graph.nodes), k=2 * N_PAIRS)
with open('pairs.txt', 'w') as file_obj:
    for source, goal in zip(nodes[:N_PAIRS], nodes[N_PAIRS:]):
        file_obj.write(f'{source} {goal}\n')
```

In [10]:
def task_boa(graph, start, goal):
    result = boa(graph, start, goal)
    result['solutions'] = [node.g for node in result['solutions']]
    result['start'] = start
    result['goal'] = goal
    
    return result


def task_namoa(graph, start, goal):
    result = namoa(graph, start, goal)
    result['start'] = start
    result['goal'] = goal
    del result['parent']
    
    return result

##  3.1 `BOA`

In [11]:
%%time
graph = read_edgelists('USA-road-d.NY.gr', 'USA-road-t.NY.gr')

start_goal_pairs = []
with open('pairs.txt', 'r') as file_obj:
    for line in file_obj:
        line = line.strip()
        if line:
            start_goal_pairs.append(tuple(int(token) for token in line.split()))


with concurrent.futures.ProcessPoolExecutor(max_workers=7) as executor:
    futures = []
    for start, goal in start_goal_pairs:
        futures.append(executor.submit(task_boa, graph, start, goal))
    
    results = [future.result() for future in concurrent.futures.as_completed(futures), total=len(futures)]    
    with open('boa_results.pickle', 'wb') as file_obj:
        pickle.dump(results, file_obj)

CPU times: user 56.7 s, sys: 1.28 s, total: 58 s
Wall time: 11min 13s


##  3.2 `NAMOA`

In [13]:
%%time
graph = read_edgelists('USA-road-d.NY.gr', 'USA-road-t.NY.gr')

start_goal_pairs = []
with open('pairs.txt', 'r') as file_obj:
    for line in file_obj:
        line = line.strip()
        if line:
            start_goal_pairs.append(tuple(int(token) for token in line.split()))


with concurrent.futures.ProcessPoolExecutor(max_workers=7) as executor:
    futures = []
    for start, goal in start_goal_pairs:
        futures.append(executor.submit(task_namoa, graph, start, goal))
    
    results = [future.result() for future in concurrent.futures.as_completed(futures), total=len(futures)]    
    with open('namoa_results.pickle', 'wb') as file_obj:
        pickle.dump(results, file_obj)

CPU times: user 58.4 s, sys: 1.44 s, total: 59.9 s
Wall time: 1h 22min 33s


##  3.3 Пример результатов

In [15]:
with open('boa_results.pickle', 'rb') as file_obj:
    boa_results = pd.DataFrame(pickle.load(file_obj))
    boa_results.set_index(keys=['start', 'goal'], inplace=True)
    boa_results.sort_index(inplace=True)

boa_results.head()

,,solutions,n_expansions,runtime,max_size
start,goal,,,,
581,205448,"[(546183, 909771), (546242, 903812), (546880, ...",26776,4.415459,3690
650,176483,"[(735979, 1306497), (736031, 1305654), (736038...",633558,25.949776,51230
4393,159868,"[(1042436, 2012628), (1042479, 2007985), (1042...",251351,10.954799,21498
15332,150787,"[(443588, 980882), (443589, 980881), (443590, ...",81778,5.236346,12044
23924,263320,"[(492751, 839015), (492754, 837314), (492809, ...",91410,4.976904,15046


In [16]:
with open('namoa_results.pickle', 'rb') as file_obj:
    namoa_results = pd.DataFrame(pickle.load(file_obj))
    namoa_results.set_index(keys=['start', 'goal'], inplace=True)
    namoa_results.sort_index(inplace=True)

namoa_results.head()

,,solutions,n_expansions,runtime,max_size,max_size_invalid
start,goal,,,,,
581,205448,"[(546183, 909771), (546242, 903812), (546880, ...",26776,3.998603,2314,3407
650,176483,"[(735979, 1306497), (736031, 1305654), (736038...",633807,110.397955,34549,44948
4393,159868,"[(1042436, 2012628), (1042479, 2007985), (1042...",251351,46.328172,12115,19007
15332,150787,"[(443588, 980882), (443589, 980881), (443590, ...",81778,50.862343,4753,10047
23924,263320,"[(492751, 839015), (492754, 837314), (492809, ...",91410,17.418110,10134,13726


## 3.4 Сравнение полученных решений

In [25]:
print(
    'Are solutions same:',
    boa_results.solutions.apply(lambda x: set(x)).equals(namoa_results.solutions.apply(lambda x: set(x)))
)

Are solutions same: True


## 3.5 Сравнение `n_expansion`

In [29]:
boa_results[boa_results['n_expansions'] != namoa_results['n_expansions']]

,,solutions,n_expansions,runtime,max_size
start,goal,,,,
650,176483,"[(735979, 1306497), (736031, 1305654), (736038...",633558,25.949776,51230


In [30]:
namoa_results[boa_results['n_expansions'] != namoa_results['n_expansions']]

,,solutions,n_expansions,runtime,max_size,max_size_invalid
start,goal,,,,,
650,176483,"[(735979, 1306497), (736031, 1305654), (736038...",633807,110.397955,34549,44948


## 3.6 Статистика для `BOA`

In [31]:
boa_results.drop(columns=['solutions']).describe(percentiles=[]).T[['min', 'mean', 'max', 'std']].T

,n_expansions,runtime,max_size
min,4.900000e+01,1.801484,28.000000
mean,4.896015e+05,27.873722,21850.900000
max,8.497269e+06,521.712259,240390.000000
std,1.300720e+06,78.389148,36845.529102


## 3.7 Статистика для `NAMOA`

In [32]:
namoa_results.drop(columns=['solutions']).describe(percentiles=[]).T[['min', 'mean', 'max', 'std']].T

,n_expansions,runtime,max_size,max_size_invalid
min,4.900000e+01,1.619263,22.000000,27.000000
mean,4.896040e+05,189.940944,13674.350000,19005.820000
max,8.497269e+06,4236.250133,149794.000000,205330.000000
std,1.300721e+06,634.725630,23689.873754,31931.085141
